In [57]:
import pandas as pd
import numpy as np

In [6]:
swap_rates = pd.read_csv("../data/market-data-swap-rates.csv")
swap_rates.head(50)

,Date,Start Date,Tenor,Swap Rate
0,2021-01-13,2022-01-13,10y,1.081333
1,2021-01-13,2022-01-13,15y,1.265960
2,2021-01-13,2022-01-13,18M,0.221625
3,2021-01-13,2022-01-13,1y,0.195474
4,2021-01-13,2022-01-13,2y,0.279370
5,2021-01-13,2022-01-13,3y,0.404284
6,2021-01-13,2022-01-13,4y,0.536341
7,2021-01-13,2022-01-13,5y,0.664396
8,2021-01-13,2022-01-13,8y,0.950975
9,2021-01-13,2022-07-13,10y,1.155757


In [85]:
swap_rates.describe()
swap_rates["Date"] = pd.to_datetime(swap_rates["Date"])
swap_rates

,Date,Start Date,Tenor,Swap Rate
0,2021-01-13,2022-01-13,10y,1.081333
1,2021-01-13,2022-01-13,15y,1.265960
2,2021-01-13,2022-01-13,18M,0.221625
3,2021-01-13,2022-01-13,1y,0.195474
4,2021-01-13,2022-01-13,2y,0.279370
...,...,...,...,...
63175,2024-01-12,2039-01-12,2y,3.424505
63176,2024-01-12,2039-01-12,3y,3.393758
63177,2024-01-12,2039-01-12,4y,3.358897
63178,2024-01-12,2039-01-12,5y,3.319531


In [8]:
swap_rates_by_today = swap_rates[swap_rates["Start Date"] < "2024-05-14"]
swap_rates_by_today


# EMA Dampling 
# 

,Date,Start Date,Tenor,Swap Rate
0,2021-01-13,2022-01-13,10y,1.081333
1,2021-01-13,2022-01-13,15y,1.265960
2,2021-01-13,2022-01-13,18M,0.221625
3,2021-01-13,2022-01-13,1y,0.195474
4,2021-01-13,2022-01-13,2y,0.279370
...,...,...,...,...
48442,2023-05-12,2024-05-13,2y,2.726265
48443,2023-05-12,2024-05-13,3y,2.669241
48444,2023-05-12,2024-05-13,4y,2.664295
48445,2023-05-12,2024-05-13,5y,2.678270


In [93]:
def ema(gp):
    #  Date is in the order natrually, so no need to rearrange the date
    N = len(gp)
    alpha = 2 / (N + 1)
    coe = []
    for i in range(N):
        coe.append(alpha * ((1 - alpha) ** i))
    coe.reverse()
    print(coe)
    return sum(np.array(gp) * np.array(coe))



In [94]:
gp_vols = swap_rates[:1000].groupby(['Start Date', 'Tenor'])['Swap Rate'].apply(ema).reset_index()
gp_vols

0    1.081333
Name: (2022-01-13, 10y), dtype: float64
[1.0]
1    1.26596
Name: (2022-01-13, 15y), dtype: float64
[1.0]
2    0.221625
Name: (2022-01-13, 18M), dtype: float64
[1.0]
3    0.195474
Name: (2022-01-13, 1y), dtype: float64
[1.0]
4    0.27937
Name: (2022-01-13, 2y), dtype: float64
[1.0]
5    0.404284
Name: (2022-01-13, 3y), dtype: float64
[1.0]
6    0.536341
Name: (2022-01-13, 4y), dtype: float64
[1.0]
7    0.664396
Name: (2022-01-13, 5y), dtype: float64
[1.0]
8    0.950975
Name: (2022-01-13, 8y), dtype: float64
[1.0]
81    1.089143
Name: (2022-01-17, 10y), dtype: float64
[1.0]
82    1.282148
Name: (2022-01-17, 15y), dtype: float64
[1.0]
83    0.20768
Name: (2022-01-17, 18M), dtype: float64
[1.0]
84    0.183757
Name: (2022-01-17, 1y), dtype: float64
[1.0]
85    0.265282
Name: (2022-01-17, 2y), dtype: float64
[1.0]
86    0.38801
Name: (2022-01-17, 3y), dtype: float64
[1.0]
87    0.520584
Name: (2022-01-17, 4y), dtype: float64
[1.0]
88    0.651427
Name: (2022-01-17, 5y), dtype: f

,Start Date,Tenor,Swap Rate
0,2022-01-13,10y,1.081333
1,2022-01-13,15y,1.265960
2,2022-01-13,18M,0.221625
3,2022-01-13,1y,0.195474
4,2022-01-13,2y,0.279370
...,...,...,...
788,2036-01-31,2y,1.697844
789,2036-01-31,3y,1.692126
790,2036-01-31,4y,1.682458
791,2036-01-31,5y,1.674234


In [46]:
swap_rates_by_today["Start Date"].value_counts()
# 根据date进行排序，用最新的startdate的swap rate

99

In [10]:
len(swap_rates)

63180

In [28]:
swap_vols = pd.read_csv("../data/market-data-swaption-vols.csv")
swap_vols["Date"].value_counts()

2022-04-08    810
2022-06-07    810
2023-10-30    810
2022-03-09    810
2021-12-09    810
             ... 
2023-09-28    405
2023-02-14    405
2021-11-19    405
2022-07-21    405
2023-02-13    405
Name: Date, Length: 754, dtype: int64

In [29]:
swap_vols.head(10)

,Date,Expiry,Tenor,Strike,Vols
0,2021-01-13,10y,10y,atm-1.0%,0.170552
1,2021-01-13,10y,10y,atm-0.5%,0.135412
2,2021-01-13,10y,10y,atm,0.114587
3,2021-01-13,10y,10y,atm+0.5%,0.137742
4,2021-01-13,10y,10y,atm+1.0%,0.169685
5,2021-01-13,10y,15y,atm-1.0%,0.156420
6,2021-01-13,10y,15y,atm-0.5%,0.127636
7,2021-01-13,10y,15y,atm,0.108229
8,2021-01-13,10y,15y,atm+0.5%,0.130351
9,2021-01-13,10y,15y,atm+1.0%,0.161753


In [47]:
len(swap_vols)

315900

In [35]:
# 按照strike和expiration分组，并计算平均波动率
gp_vols = swap_vols.groupby(['Expiry', 'Tenor', 'Date'])['Vols'].mean().reset_index()
print("\nAggregated DataFrame (mean volatility):")
print(gp_vols)



Aggregated DataFrame (mean volatility):
      Expiry Tenor        Date      Vols
0        10y   10y  2021-01-13  0.145596
1        10y   10y  2021-01-15  0.145449
2        10y   10y  2021-01-18  0.145109
3        10y   10y  2021-01-19  0.144115
4        10y   10y  2021-01-20  0.143333
...      ...   ...         ...       ...
61069     8y    8y  2024-01-08  0.256855
61070     8y    8y  2024-01-09  0.256540
61071     8y    8y  2024-01-10  0.256868
61072     8y    8y  2024-01-11  0.258929
61073     8y    8y  2024-01-12  0.261329

[61074 rows x 4 columns]


In [12]:
swap_vols["Expiry"].value_counts()

10y    35100
3y     35100
5y     35100
15y    35100
4y     35100
8y     35100
1y     35100
18M    35100
2y     35100
Name: Expiry, dtype: int64

In [13]:
swap_vols.describe()

,Vols
count,315900.000000
mean,0.380107
std,0.432522
min,0.005489
25%,0.119821
50%,0.202674
75%,0.426787
max,3.338007


In [14]:
len(swap_vols)

315900

In [15]:
trade_information = pd.read_csv("../data/trade-information.csv")
trade_information.head(20)

,trade name,underlying,pay_frequency,maturity,lower_bound,upper_bound
0,dummyTrade1,USD: CMS:2Y,6M,5Y,0.0042,0.0379
1,dummyTrade2,USD: CMS:2Y,4M,5Y,0.0042,0.0379
2,dummyTrade3,USD: CMS:5Y,6M,5Y,0.0044,0.0397
3,dummyTrade4,USD: CMS:5Y,4M,5Y,0.0044,0.0397
4,dummyTrade5,USD: CMS:10Y,4M,5Y,0.0046,0.0417
5,dummyTrade6,USD: CMS:10Y,3M,5Y,0.0046,0.0417
6,dummyTrade7,USD: CMS:2Y,6M,10Y,0.0042,0.0379
7,dummyTrade8,USD: CMS:2Y,4M,10Y,0.0042,0.0379
8,dummyTrade9,USD: CMS:5Y,6M,10Y,0.0044,0.0397
9,dummyTrade10,USD: CMS:5Y,4M,10Y,0.0044,0.0397


In [16]:
len(trade_information)

12

In [19]:
vegas = pd.read_csv("../data/trade-price-ir-vegas.csv")
vegas.head(20)


,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687
5,2022-09-02,dummyTrade1,USD,0,-215402.656152,1y,2023-09-04,10y,5.016274
6,2022-09-02,dummyTrade1,USD,5,-214640.494071,1y,2023-09-04,10y,3.458172
7,2022-09-02,dummyTrade1,USD,10,-213858.452424,1y,2023-09-04,10y,3.132444
8,2022-09-02,dummyTrade1,USD,25,-211345.804723,1y,2023-09-04,10y,12.184058
9,2022-09-02,dummyTrade1,USD,50,-206384.346001,1y,2023-09-04,10y,15.648409


In [20]:
correlation_AB = vegas['TV'].corr(vegas['Vega'])
correlation_AB

-0.003990306529211773

In [21]:
vegas["Value Date"].value_counts()

2022-11-18    11664
2023-08-31    11664
2023-12-08    11664
2023-05-24    11664
2023-11-14    11664
              ...  
2022-12-08    11664
2022-11-02    11664
2023-10-02    11664
2023-04-17    11664
2022-11-16    11664
Name: Value Date, Length: 336, dtype: int64

In [22]:
vegas["Tenor Bucket"].value_counts()

10y    435456
1y     435456
2y     435456
18M    435456
3y     435456
5y     435456
15y    435456
4y     435456
8y     435456
Name: Tenor Bucket, dtype: int64

In [23]:
435456/12
# EACH TRADE HAS 36288 different tenors 36288 15y tenors

36288.0

In [24]:
len(vegas[vegas["Trade Name"] == "dummyTrade1"])

326592

In [25]:
vegas.describe()

,Zero Rate Shock,TV,Vega
count,3.919104e+06,3.919104e+06,3.919104e+06
mean,6.250000e+00,-3.471395e+05,1.714511e+01
std,5.136167e+01,1.856180e+05,2.850144e+03
min,-1.000000e+02,-7.174312e+05,-1.385180e+06
25%,-1.375000e+01,-5.291890e+05,-1.342167e+00
50%,2.500000e+00,-2.242142e+05,1.336000e-04
75%,3.125000e+01,-1.857137e+05,7.386112e+00
max,1.000000e+02,-2.911344e+04,1.110263e+06


In [26]:
len(vegas)

3919104

# Data Combination of vegas sheet and the volities sheet

In [41]:
vegas.head(10)

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687
5,2022-09-02,dummyTrade1,USD,0,-215402.656152,1y,2023-09-04,10y,5.016274
6,2022-09-02,dummyTrade1,USD,5,-214640.494071,1y,2023-09-04,10y,3.458172
7,2022-09-02,dummyTrade1,USD,10,-213858.452424,1y,2023-09-04,10y,3.132444
8,2022-09-02,dummyTrade1,USD,25,-211345.804723,1y,2023-09-04,10y,12.184058
9,2022-09-02,dummyTrade1,USD,50,-206384.346001,1y,2023-09-04,10y,15.648409


In [45]:
V_vegas = pd.merge(vegas.head(100), gp_vols, left_on = ["Value Date", "Expiry Bucket", "Tenor Bucket"], \
                right_on= ["Date", "Expiry", "Tenor"], how = "inner")

V_vegas = V_vegas.drop(columns = ['Expiry', "Tenor", "Date"])
V_vegas

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,Vols
0,2022-09-02,dummyTrade1,USD,-100,-227907.098775,1y,2023-09-04,10y,1.962246,0.223827
1,2022-09-02,dummyTrade1,USD,-50,-222208.400967,1y,2023-09-04,10y,-3.812341,0.223827
2,2022-09-02,dummyTrade1,USD,-25,-218960.927995,1y,2023-09-04,10y,4.471006,0.223827
3,2022-09-02,dummyTrade1,USD,-10,-216872.430106,1y,2023-09-04,10y,4.333398,0.223827
4,2022-09-02,dummyTrade1,USD,-5,-216146.310328,1y,2023-09-04,10y,5.679687,0.223827
...,...,...,...,...,...,...,...,...,...,...
95,2022-09-02,dummyTrade5,USD,100,-182298.810410,1y,2023-09-04,10y,-8.537942,0.223827
96,2022-09-02,dummyTrade6,USD,-100,-224125.131298,1y,2023-09-04,10y,-6.834734,0.223827
97,2022-09-02,dummyTrade6,USD,-50,-219636.077304,1y,2023-09-04,10y,-5.563973,0.223827
98,2022-09-02,dummyTrade6,USD,-25,-216429.777528,1y,2023-09-04,10y,-5.522299,0.223827
